In [ ]:
from imsciences import *

ims = dataprocessing()

path = ims.get_wd_levels(0)
path

In [ ]:
df = ims.read_and_concatenate_files(path, "csv")

In [ ]:
df

,Action,Time,ISIN,Ticker,Name,No. of shares,Price / share,Currency (Price / share),Exchange rate,Total,Currency (Total),Withholding tax,Currency (Withholding tax)
0,Dividend (Dividend),2025-05-30 12:53:12,US55354G1004,MSCI,MSCI,3.00,1.530000,USD,Not available,3.41,GBP,0.81,USD
1,Dividend (Dividend),2025-05-30 12:55:22,IE00BYXVWC37,STHS,PIMCO US Short-Term High Yield Corporate Bond ...,225.00,0.050164,GBP,Not available,11.29,GBP,0.00,GBP
2,Dividend (Dividend),2025-06-06 12:55:43,US7010941042,PH,Parker-Hannifin,1.00,1.530000,USD,Not available,1.13,GBP,0.27,USD
3,Dividend (Dividend),2025-06-23 14:31:10,GB00BP6MXD84,SHEL,Shell,75.00,26.410000,GBX,Not available,19.81,GBP,0.00,GBX
4,Dividend (Dividend),2025-06-04 10:27:35,IE00BZ163L38,VEMT,Vanguard USD Emerging Markets Government Bond ...,60.00,0.185315,GBP,Not available,11.12,GBP,-0.00,GBP
...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,Dividend (Dividend),2025-05-06 11:59:56,NL0010273215,ASML,ASML,5.25,1.564000,EUR,Not available,6.99,GBP,1.45,EUR
386,Dividend (Dividend),2025-05-07 13:22:27,IE0003UVYC20,JEGP,JPMorgan Global Equity Premium Income Active (...,80.00,13.422869,GBX,Not available,10.74,GBP,0.00,GBX
387,Dividend (Dividend),2025-05-07 13:26:59,FR0000052292,RMS,Hermes International,0.25,7.500000,EUR,Not available,1.60,GBP,0.63,EUR
388,Dividend (Dividend),2025-05-07 13:33:43,FR0000052292,RMS,Hermes International,0.25,9.375000,EUR,Not available,1.99,GBP,0.78,EUR


In [ ]:
filename = "dividends_up_to_oct_2025.csv"
df.to_csv(filename, index=False)